# 1. Preparing dataset

- 변수 정보
    - Passengerld: 승객번호
    - Survived: 생존여부 (0: 사망, 1: 생존) 
    - Pclass: 배의 클래스(1>2>3) 
    - Name: 승객이름
    - Sex: 성별
    - Age: 승객 나이 
    - SibSp: 형제, 자매, 배우자의 수 
    - Parch: 부모, 자녀 수 
    - Ticket: 티켓번호 
    - Fare: 요금 
    - Cabin: 객실번호 
    - Embarked: 출발지 (C: 셰르부르, S: 사우샘프턴, Q: 퀸즈타운)

In [1]:
import warnings
warnings.filterwarnings("ignore")

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import LabelEncoder, OneHotEncoder, StandardScaler, MinMaxScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score, roc_curve, auc, mean_squared_error, classification_report
from sklearn import  model_selection
from sklearn import ensemble
from sklearn import svm
from sklearn.model_selection import GridSearchCV
from skopt import BayesSearchCV
from vecstack import stacking, StackingTransformer
from xgboost import XGBClassifier
from IPython.display import Image
from sklearn.tree import DecisionTreeClassifier
from skopt import BayesSearchCV
from skopt.space import Categorical, Integer
from sklearn.linear_model import LogisticRegression

plt.rcParams['font.family'] = 'Nanum Gothic'

In [ ]:
Image('/data/workspace_files/titanic_img.png')

In [ ]:
# titanic dataframe 생성
titan_df = pd.read_csv('/data/workspace_files/titanic.csv',)

In [ ]:
# data 확인 
titan_df.head(5)

In [ ]:
# titan dataframe infomation
titan_df.info()

- 변수 분류
    - Categorical(범주형): Survived, Sex, and Embarked. Ordinal: Pclass.
    - Continous(연속형): Age, Fare
    - Discrete(이산형): SibSp, Parch
> 7개의 변수는 integer or floats / 5개의 변수는 strings (object).

In [ ]:
# numerical 변수 기술통계
titan_df.describe()

- train data의 표본 샘플은 타이나닉호에 탑승한 실제 승객 수(2,224명)의 40%인 891명이다. 
- 표본 샘플의 생존율은 약 38%이다. 
- 대부분의 승객(75% 이상)은 부모나 자녀와 함께 탑승하지 않았다. 
- 지불 요금은 상당히 다양하게 나타나며, 512달러 만큼 높은 요금을 지불하는 경우는 거의 없었다.(1% 미만) 
- 65-80세 범위에 속하는 노인은 거의 없다. (1% 미만)

In [ ]:
# Categorical 변수 기술통계 
titan_df.describe(include=['O'])

- 이름은 중복값이 존재하지 않는 고유값이다. (freq = 1)
- 승객 중 65%는 남성이었다. (577/891)
- 객실번호에 중복값이 다수인 것으로 보아, 여러명의 승객이 객실을 공유한 것으로 파악된다. (Cabin)
- 대부분의 승객이 사우샘프턴에서 탑승했다. (Embarked=S)
- 티겟번호 또한 중복 비율이 높다.(23%) (210/891)

In [ ]:
# 결측치 확인 
titan_df.isna().sum()

- Age: 177개의 결측치 존재 
- Cabin: 687개의 결측치 존재 
- Embarked: 2개의 결측치 존재 

#### Cabin > Age > Embarked 순으로 결측치 존재

In [ ]:
Image('/data/workspace_files/titanic_structure.png')

### PassengerId, Cabin, Name, Ticket 열 제거 
1. Cabin: 위 사진에서 A-P 번호는 각 객실번호(Cabin)의 앞자리에 붙은 객실 구분 번호를 뜻함. 하지만 각 객실번호가 어떤 Pclass에 속하는 지 파악 불가하며 여러명의 승객이 같은 객실을 공유했다는 점에서 이것이 생존여부에 미친 영향을 확인하기 어렵다고 판단 
  
2. Name: Mr, Miss와 같은 호칭을 통해서 승객들의 성별, 연령대를 파악할 수 있으나 이러한 정보는 이미 Age, Sex 컬럼에서 제시하고 있으므로 삭제
    
3. Ticket: 중복된 값이 다수 존재함에 따라 Ticket 번호가 생존여부에 미친 영향을 파악하기 어려울 것이라 판단 후 삭제 
  
> 따라서 해당 컬럼들을 제외하고 Pclass, Sex, Age, SibSp, Parch, Fare, Embarked 변수와 우리가 예측하고자 하는 값인 Survived 변수 간에 어떠한 관계가 있는 지 시각화 후 모델링

# 2. Feature engineering & Feature selection

- #### 시각화를 통해 각 독립변수가 종속변수(생존여부) 간의 관계 파악 

#### 시도해볼 수 있는 전략들

- 불필요한 열이나 예측에 방해가 되는 열은 아예 지우기 (ex. PassengerId)
- 결측치 채우기 
- Text로 되어있는 Category(Factor)는 숫자로 바꿔주기 (ex. Male/Female -> 0/1)
- 실수 범위를 구간 범위로 바꿔주기 
- 필요한 경우 기존 열을 바탕으로 새로운 열을 계산해 추가하기

ex. 보험사에서 다루는 고객데이터셋 중 나이열에 있는 결측치를 다루려 할 때
- 랜덤 값으로 채우기
- 대표 값으로 채우기 (mean평균, median중위값, mode최빈값)
- 위아래 데이터의 평균값으로 채우기
- 행들을 그룹으로 묶은 후 해당 그룹의 대표값으로 채우기
- 머신러닝 예측모델을 만들어 해당 열의 결측치를 채우기
- 특정 기준 이상의 결측치가 있을 시 해당 열 삭제

In [ ]:
# 데이터 탐색을 위한 원본 데이터 70:30 비율로 split
train_df, test_df = model_selection.train_test_split(titan_df, test_size=0.3, random_state=0)

### Pclass

In [ ]:
# 배의 클래스(1>2>3)
train_df.Pclass.value_counts()

In [ ]:
sns.violinplot( x=train_df['Survived'], y=train_df['Pclass'])
plt.show()

In [ ]:
sns.barplot(x=train_df.Pclass, y=train_df.Survived)
plt.show()

In [ ]:
sns.barplot(x='Pclass', y='Survived', hue='Sex', data=train_df)
plt.show()

### Age

In [ ]:
# 승객 연령 분포 (20-40대 사이가 가장 많음)
train_df.plot(kind="hist", y="Age", bins=50)
plt.show()

In [ ]:
# Age컬럼의 기술 통계
train_df.Age.describe()

In [ ]:
# 생존여부 별 승객 연령 분포 확인

age_histo = sns.FacetGrid(train_df, col='Survived')
age_histo.map(plt.hist, 'Age', bins=20)
plt.show()

In [ ]:
# Pclass 및 생존여부 별 연령대 분포 파악 
age_histo = sns.FacetGrid(train_df, row='Pclass',col='Survived')
age_histo.map(plt.hist, 'Age', bins=20)
plt.show()

### Sex

In [ ]:
# 성별에 따른 생존률
plt.figure(figsize=(8,6))
sns.barplot(train_df.Sex, train_df.Survived)
plt.show()

In [ ]:
plt.figure(figsize=(8,6))
plt.plot(aspect='auto')
plt.pie(titan_df['Sex'].value_counts(sort=False), labels=train_df['Sex'].value_counts(sort=False).index, autopct='%1.1f%%')
plt.title('Gender ratio')
plt.show()

### Fare

In [ ]:
# fare
train_df.plot(kind="hist", y="Fare", bins=50)
plt.show()

### SibSp & Parch

In [ ]:
sns.barplot(train_df.SibSp, train_df.Survived)
plt.show()

In [ ]:
sns.barplot(train_df.Parch, train_df.Survived)
plt.show()

In [ ]:
# 형제, 자매, 배우자의 수 + 부모, 자녀의 수를 합쳐 파생컬럼 생성 
titan_test = titan_df
titan_test['family'] = titan_test.SibSp + titan_df.Parch

In [ ]:
sns.barplot(titan_test.family, titan_test.Survived)
plt.show()

### Embarked

In [ ]:
# 어디서 탔는 지에 따라 생존률이 달라질까? 
embarked_df = pd.DataFrame(round(titan_df.groupby(['Embarked'])['Survived'].value_counts(normalize=True), 2))
embarked_df

In [ ]:
# 승선지에 따라서 Pclass별로 여성 및 남성의 생존율이 다르게 나타날까? 
grid = sns.FacetGrid(train_df, row='Embarked', size=2.2, aspect=1.6)
grid.map(sns.pointplot, 'Pclass', 'Survived', 'Sex')
grid.add_legend()

### 변수 간 상관관계 분석

1. 종속변수와 독립변수 간 상관관계 분석 
- 범주형-범주형: Survived-Pclass, Embarked -> Phi Correlation (각 범주 변수를 0과 1로 바꾼 다음, 이 둘 간의 Pearson 상관계수로 계산)
- 범주형-연속형: Survived-Age, SibSp, Parch -> Point biserial Correlation (이분변수를 0과 1로 코딩한 다음 Peason 상관계수를 계산)

2. 독립변수 간 상관관계 분석
- 범주형: Pclass, Embarked
- 연속형: Age, SibSp, Parch 
-> 범주형, 연속형 변수 섞여있으므로 Point biserial Correlation 

<br>

# 2. Train - Test split (비율을 7:3 으로 유지해주시고, seed는 0을 적용해주세요)

In [ ]:
# Age에 랜덤하게 백분위수 25%~75%까지의 값을 랜덤하게 대치하기 위한 함수
def replace_age(age_quantile_25, age_quantile_75):
    return np.random.randint(age_percentiles_25, age_percentiles_75)

In [ ]:
# Age의 백분위수 25%
age_percentiles_25 = titan_df.Age.quantile(.25)
# Age의 백분위수 75%
age_percentiles_75 = titan_df.Age.quantile(.75)

# Age에 랜덤하게 백분위수 25%~75%까지의 값을 랜덤하게 대치
titan_df.Age = titan_df.Age.apply(lambda x : replace_age(age_percentiles_25, age_percentiles_75) if pd.isna(x) else x)

In [ ]:
# Embarked에 2개의 결측치 제거
titan_df.dropna(subset=['Embarked'], inplace=True)

In [ ]:
# 필요없는 컬럼 제거
titan_df.drop(columns=['Cabin', 'Ticket', 'Name', 'PassengerId'], inplace=True)

# target(Survived) column만 나눠놓은 것
target_df = titan_df['Survived']
# Survived column을 지우고 실제로 train_split 할 데이터
data_df = titan_df.drop(columns=['Survived'])


In [ ]:
titan_df.isna().sum()

In [ ]:
# 7:3의 비율, seed는 0으로 데이터를 train과 test로 나누는 과정
x_train, x_test, y_train, y_test = model_selection.train_test_split(data_df, target_df,
                                                                    stratify=target_df,
                                                                    test_size=0.3,
                                                                    random_state=42)

In [ ]:
# 연속형 데이터
numeric_features = ['SibSp', 'Parch', 'Age', 'Fare']
# 연속형 데이터 encoder
numeric_transformer = StandardScaler()
# 범주형 데이터
categorical_features = ['Sex', 'Pclass', 'Embarked']
# 범주형 데이터 encoder
categorical_transformer = OneHotEncoder(categories='auto', handle_unknown='ignore') 

preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features)],
)

In [ ]:
preprocessor_pipe = Pipeline(steps=[('preprocessor', preprocessor)])

In [ ]:
# x_train데이터 학습시키는 과정
preprocessor_pipe.fit(x_train)

In [ ]:
# x_train의 학습을 토대로 One-HotEncoding(), StandardScaler() 적용
x_train_transformed = preprocessor_pipe.transform(x_train)
x_test_transformed = preprocessor_pipe.transform(x_test)

In [ ]:
# 컬럼명 변경
x_train_encoding_df = pd.DataFrame(x_train_transformed, columns=['SibSp', 'Parch', 'Age', 'Fare', 'Female', 'Male', 'Pclass_1', 'Pclass_2', 'Pclass_3', 'Embarked_C', 'Embarked_Q', 'Embarked_S'])
x_test_encoding_df =  pd.DataFrame(x_test_transformed, columns=['SibSp', 'Parch', 'Age', 'Fare', 'Female', 'Male', 'Pclass_1', 'Pclass_2', 'Pclass_3', 'Embarked_C', 'Embarked_Q', 'Embarked_S'])


<br>

# 3. Train the model


## Logistic Regression

## Support Vector Machine
HPO(Hyper-parameter optimization) : GridSearch 사용


## Stacking에 들어간 모델들
1. ExtraTrees
2. RandomForest
3. XGB
4. DecisionTree
5. LogisticReression
6. SupportVectorMachine

## XG Boost - Gradient Boosting

### 1. Gradient Boosting regression

#### 파생컬럼(Family 미포함)
- accuracy: 
- 
#### 파생컬럼(Family 포함)


### 2. Gradient Boosting classification

# 4. 향후 과제 